In [1]:
from symmer.chem.chem_utils import xyz_from_pubchem
from symmer.chem.fermionic_ham import *
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator

/home/tim/anaconda3/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
2022-05-30 15:18:40.197943: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-30 15:18:40.496704: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-30 15:18:40.528140: W external/org_tensorflow/tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [4]:
mol_name = 'Al'
# mol_name = 'H2'

# mol_name = 'LiH'

In [5]:
xyz_file = xyz_from_pubchem(mol_name)
print(xyz_file)

Unable to find structure info in the PubChem databasefor the specified molecule Al.
1
 
Al	2	0	0



In [7]:
from symmer.chem.chem_utils import Draw_molecule

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
basis = 'STO-3G'
convergence = 1e-6
charge=+1
max_hf_cycles=500
ram = 8_000
run_mp2  = True,
run_cisd = True,
run_ccsd = True,
run_fci  = True

In [15]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [16]:
pyscf_obj.run_pyscf()

In [17]:
pyscf_obj.pyscf_hf.e_tot

-238.88005288340423

In [18]:
2*pyscf_obj.pyscf_hf.mol.nao

18

In [19]:
from symmer.chem.fermionic_ham import FermionicHamilt

In [20]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_fermionic_hamiltonian_operator()

H_ferm.fermionic_molecular_hamiltonian

() 0
((0, 1), (0, 0)) -83.50683297651027
((0, 1), (2, 0)) 1.4441705679192085
((0, 1), (10, 0)) -0.4041828014017632
((1, 1), (1, 0)) -83.50683297651027
((1, 1), (3, 0)) 1.4441705679192085
((1, 1), (11, 0)) -0.4041828014017632
((2, 1), (0, 0)) 1.4441705679192045
((2, 1), (2, 0)) -19.754437204877366
((2, 1), (10, 0)) 1.7263311902879954
((3, 1), (1, 0)) 1.4441705679192045
((3, 1), (3, 0)) -19.754437204877366
((3, 1), (11, 0)) 1.7263311902879954
((4, 1), (4, 0)) -18.45680932964686
((4, 1), (12, 0)) 2.3396888317474644
((4, 1), (14, 0)) 0.0017469455086113128
((4, 1), (16, 0)) 0.05028126725909212
((5, 1), (5, 0)) -18.45680932964686
((5, 1), (13, 0)) 2.3396888317474644
((5, 1), (15, 0)) 0.0017469455086113128
((5, 1), (17, 0)) 0.05028126725909212
((6, 1), (6, 0)) -18.45680932964687
((6, 1), (12, 0)) -0.049279467304560426
((6, 1), (14, 0)) -0.3917376602675079
((6, 1), (16, 0)) 2.3066833824107653
((7, 1), (7, 0)) -18.45680932964687
((7, 1), (13, 0)) -0.049279467304560426
((7, 1), (15, 0)) -0.39173

In [22]:
mol_data = molecule_geometries[speciesname]
#atoms  = mol_data['atoms']
#coords = mol_data['coords']
geometry = [(('Al', 0,0,0))] #list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=+1, basis='STO-3G', spin=0, run_fci=True, print_info=False)

# taper the Hamiltonian
taper_hamiltonian = QubitTapering(molecule.H_q)
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
hf_tapered = taper_hamiltonian.tapered_ref_state

# perform legacy CS-VQE calculations
H = ham_tap.to_dictionary
H_noncon = greedy_dfs(H, cutoff=10)[-1]
H_noncon = {P:H[P] for P in H_noncon}
nc_legacy, energies_legacy, errors_legacy, order_legacy = csvqe_approximations_heuristic(
    H, H_noncon, ham_tap.n_qubits, molecule.fci_energy)

# initiate stabilizer identification classes
Hc_stabilizers = StabilizerIdentification(ham_tap)
CC_stabilizers = StabilizerIdentification(ucc_tap)
HL_gap = np.where(hf_tapered==0)[0][0]-.5
ham_biasing = ObservableBiasing(base_operator=ham_tap, HOMO_LUMO_gap=HL_gap)

# build CS-VQE model
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap, noncontextual_form='diag')


NameError: name 'molecule_geometries' is not defined

In [21]:
H_ferm.fermionic_fock_operator

-58.13558472935034 [0^ 0] +
-58.13558472935034 [1^ 1] +
-5.060459710269678 [2^ 2] +
-5.060459710269678 [3^ 3] +
-3.2724768872482386 [4^ 4] +
-3.2724768872482386 [5^ 5] +
-3.272476887248235 [6^ 6] +
-3.272476887248235 [7^ 7] +
-3.272476887248228 [8^ 8] +
-3.272476887248228 [9^ 9] +
-0.6022046545777302 [10^ 10] +
-0.6022046545777302 [11^ 11] +
0.023271628624167207 [12^ 12] +
0.023271628624167207 [13^ 13] +
0.023271628624168983 [14^ 14] +
0.023271628624168983 [15^ 15] +
0.023271628624168095 [16^ 16] +
0.023271628624168095 [17^ 17]

In [13]:
H_ferm.scf_method.mo_energy

array([-20.2439146 ,  -1.26308588,  -0.61087371,  -0.4528697 ,
        -0.3909081 ,   0.59496821,   0.72693967])

In [14]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [15]:
hf_ket = H_ferm.hf_ket
hf_ket.shape

(16384,)

In [16]:
## check
from functools import reduce
state = {0: np.array([[1],[0]]), 1: np.array([[0],[1]])}
ll = [state[bit] for bit in hf_state]
kket = reduce(np.kron, ll)
all(kket[:,0] == hf_ket)

True

In [17]:
print(H_ferm.scf_method.energy_tot())
print(pyscf_obj.pyscf_hf.energy_tot())

H_mat = H_ferm.get_sparse_ham()
hf_ket.conj().T @ H_mat @ hf_ket

-74.96444758276998
-74.96444758276998


(-74.96444758276998+0j)

In [18]:
one_body_integrals = H_ferm._one_body_integrals
two_body_integrals = H_ferm._two_body_integrals

g = two_body_integrals
fij = pyscf_obj.pyscf_hf.mo_energy

nocc = H_ferm.n_electrons // 2  # this is never the active space
ei = fij[:nocc]
ai = fij[nocc:]
abgij = g[nocc:, nocc:, :nocc, :nocc]
amplitudes = abgij * 1.0 / (
        ei.reshape(1, 1, -1, 1) + ei.reshape(1, 1, 1, -1) - ai.reshape(-1, 1, 1, 1) - ai.reshape(1, -1, 1, 1))
E = 2.0 * np.einsum('abij,abij->', amplitudes, abgij) - np.einsum('abji,abij', amplitudes, abgij,
                                                                        optimize='greedy')

E

-0.03655064467197491

In [19]:
pyscf_obj.pyscf_mp2.e_corr

-0.03655064467197491

In [20]:
T2_mp2 = get_T2_mp2(pyscf_obj.pyscf_mp2.t2)
T2_mp2

-0.000185301319319176 [10^ 0 11^ 1] +
-7.86528174073248e-05 [10^ 0 11^ 3] +
5.6993432638427695e-06 [10^ 0 11^ 7] +
0.00012072280269846382 [10^ 0 12^ 4] +
7.77343162487925e-05 [10^ 0 13^ 5] +
0.000185301319319176 [10^ 1 11^ 0] +
7.865281740732526e-05 [10^ 1 11^ 2] +
-5.6993432638430304e-06 [10^ 1 11^ 6] +
4.298848644967127e-05 [10^ 1 13^ 4] +
-7.865281740732526e-05 [10^ 2 11^ 1] +
-0.006824962692070313 [10^ 2 11^ 3] +
-3.293731090601111e-08 [10^ 2 11^ 5] +
-0.005248400877835878 [10^ 2 11^ 7] +
-0.003217909076913856 [10^ 2 12^ 4] +
1.2282099799057175e-08 [10^ 2 13^ 3] +
-0.0059571718169397665 [10^ 2 13^ 5] +
2.2154717953534504e-08 [10^ 2 13^ 7] +
7.86528174073248e-05 [10^ 3 11^ 0] +
0.006824962692070313 [10^ 3 11^ 2] +
3.2937310907709655e-08 [10^ 3 11^ 4] +
0.005248400877835885 [10^ 3 11^ 6] +
-1.2282099799057175e-08 [10^ 3 13^ 2] +
0.0027392627400259087 [10^ 3 13^ 4] +
-2.5440646268044558e-08 [10^ 3 13^ 6] +
-3.2937310907709655e-08 [10^ 4 11^ 3] +
-0.007359670736816732 [10^ 4 11^ 5] +
-

In [21]:
V_pert = H_ferm.get_perturbation_correlation_potential()

In [22]:
T2_mp2_mat = get_sparse_operator(T2_mp2, n_qubits= H_ferm.n_qubits)
mp2_state = T2_mp2_mat @ hf_ket

In [23]:
hf_ket.conj().T @ V_pert @ mp2_state 

(-0.036550644671969695+0j)

In [24]:
pyscf_obj.pyscf_mp2.e_corr

-0.03655064467197491

In [25]:
hf_ket.conj().T @ V_pert @ mp2_state  - pyscf_obj.pyscf_mp2.e_corr

(5.218048215738236e-15+0j)

In [28]:
(mp2_state.conj().T  @ hf_ket) # "intermediate normalization!" (Szabo pg 323)

0j

In [29]:
pyscf_obj.pyscf_mp2.e_corr

-0.03655064467197491

In [33]:
(hf_ket.conj().T @ H_mat @ mp2_state + pyscf_obj.pyscf_mp2.e_hf).real

-75.00099822744195

In [34]:
pyscf_obj.pyscf_mp2.e_tot

-75.00099822744195